In [1]:
import sys
sys.path.append("/home/hmora/network-gen-pipeline/src")

import utils.DateUtils as du

import duckdb
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
from geopandas import datasets, GeoDataFrame, read_file, GeoSeries
from pandas import concat

from shapely.geometry import shape
import shapely.wkt

import json

import os
from pathlib import Path

from dotenv import load_dotenv
import os
load_dotenv()

duck = duckdb.connect()

/tmp/ipykernel_1889750/2164453861.py:8: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [2]:
YEAR, MONTH, DAY = ("2020", "01", "07")

### Obtención de ZMV

In [3]:
zonas_metropolitanas_vm = duck.sql(f"""
WITH
all_zones AS (
    SELECT *
    FROM read_csv_auto('{os.environ["ZONAS_METROPOLITANAS_2015"]}')
)

SELECT CVE_ZM AS cve_zm
    , CVE_MUN AS cve_mun
FROM all_zones
WHERE CVE_ZM = '09.01'
""").df()

In [4]:
mun_catalog = duck.sql(f"""
SELECT CONCAT(cve_agee, cve_agem) AS cve_mun
    , geometry
FROM read_parquet('{os.environ['AGEB_CATALOG']}')
""").df()

In [5]:
ZVM = duck.sql("""
WITH
pre AS (
    SELECT a.cve_zm, a.cve_mun, b.geometry
    FROM 
        zonas_metropolitanas_vm AS a
        LEFT JOIN
        mun_catalog AS b
        ON a.cve_mun = b.cve_mun
)

SELECT *
FROM pre
""").df()
ZVM["geometry"] = ZVM["geometry"].apply(lambda x: shape(json.loads(x)))

In [6]:
ZVM

,cve_zm,cve_mun,geometry
0,09.01,09016,"MULTIPOLYGON (((-99.21122917 19.4298056, -99.2..."
1,09.01,09015,"MULTIPOLYGON (((-99.16404172 19.4060894, -99.1..."
2,09.01,15109,"MULTIPOLYGON (((-99.18728388 19.58904478, -99...."
3,09.01,09005,"MULTIPOLYGON (((-99.08879198 19.47500166, -99...."
4,09.01,15031,"MULTIPOLYGON (((-98.95457178 19.41445346, -98...."
...,...,...,...
6053,09.01,09007,"MULTIPOLYGON (((-99.05376251 19.347039801, -99..."
6054,09.01,09007,"MULTIPOLYGON (((-98.99044098 19.34507778, -98...."
6055,09.01,09014,"MULTIPOLYGON (((-99.18465445 19.37473385, -99...."
6056,09.01,09014,"MULTIPOLYGON (((-99.132397819 19.3911572, -99...."


In [7]:
assert len(ZVM.cve_mun.unique()) == len(zonas_metropolitanas_vm["cve_mun"])
assert (ZVM[ZVM["geometry"].isnull()]).empty

### Lectura de pings en 2020-01-07

In [8]:
dates = du.trusted_range(YEAR, MONTH, DAY, 10)
dates

2020-01-02 <--2020-01-07--> 2020-01-12


[datetime.datetime(2020, 1, 2, 0, 0),
 datetime.datetime(2020, 1, 3, 0, 0),
 datetime.datetime(2020, 1, 4, 0, 0),
 datetime.datetime(2020, 1, 5, 0, 0),
 datetime.datetime(2020, 1, 6, 0, 0),
 datetime.datetime(2020, 1, 7, 0, 0),
 datetime.datetime(2020, 1, 8, 0, 0),
 datetime.datetime(2020, 1, 9, 0, 0),
 datetime.datetime(2020, 1, 10, 0, 0),
 datetime.datetime(2020, 1, 11, 0, 0),
 datetime.datetime(2020, 1, 12, 0, 0)]

In [9]:
dfs = []

for d in dates:
    curr_path = \
        f"{os.environ[f'MOVILIDAD_RAW_{YEAR}']}/month={str(d.month).zfill(2)}/day={str(d.day).zfill(2)}"

    print(curr_path)

    result = duck.sql(f"""
    WITH
    raw_data AS (
        SELECT *
        FROM read_parquet('{curr_path}/*')
    )

    , with_cdmx_datetime AS (
        SELECT *
            , TIMEZONE('America/Mexico_City', TO_TIMESTAMP(utc_timestamp)) AS cdmx_datetime
        FROM raw_data
    )

    SELECT utc_timestamp, cdmx_datetime, caid
        , latitude, longitude, horizontal_accuracy
    FROM with_cdmx_datetime
    WHERE STRFTIME(cdmx_datetime, '%Y-%m-%d') = '{YEAR}-{MONTH}-{DAY}'
        AND horizontal_accuracy >= 100
    """).df()

    if not result.empty:
        dfs.append(result)

big_union = pd.concat(dfs)
big_union = big_union.drop_duplicates()

/datos/botanero/botanero/aws/movement_parquet/year=2020/month=01/day=02
/datos/botanero/botanero/aws/movement_parquet/year=2020/month=01/day=03
/datos/botanero/botanero/aws/movement_parquet/year=2020/month=01/day=04
/datos/botanero/botanero/aws/movement_parquet/year=2020/month=01/day=05
/datos/botanero/botanero/aws/movement_parquet/year=2020/month=01/day=06
/datos/botanero/botanero/aws/movement_parquet/year=2020/month=01/day=07
/datos/botanero/botanero/aws/movement_parquet/year=2020/month=01/day=08
/datos/botanero/botanero/aws/movement_parquet/year=2020/month=01/day=09
/datos/botanero/botanero/aws/movement_parquet/year=2020/month=01/day=10
/datos/botanero/botanero/aws/movement_parquet/year=2020/month=01/day=11
/datos/botanero/botanero/aws/movement_parquet/year=2020/month=01/day=12


In [10]:
big_union

,utc_timestamp,cdmx_datetime,caid,latitude,longitude,horizontal_accuracy
0,1578368282,2020-01-07 09:38:02+00:00,212a38f3fc9aa200477b03b31bb0a4b55d08a2652e294e...,21.052160,-89.630840,112.000000
1,1578368331,2020-01-07 09:38:51+00:00,212a38f3fc9aa200477b03b31bb0a4b55d08a2652e294e...,21.052150,-89.630830,112.000000
2,1578371330,2020-01-07 10:28:50+00:00,d66445e78dfe7c118770a1b013723360992896d8bea435...,21.053164,-89.630510,142.000000
3,1578371333,2020-01-07 10:28:53+00:00,d66445e78dfe7c118770a1b013723360992896d8bea435...,21.053164,-89.630510,142.500000
4,1578371337,2020-01-07 10:28:57+00:00,d66445e78dfe7c118770a1b013723360992896d8bea435...,21.053164,-89.630510,142.000000
...,...,...,...,...,...,...
1310042,1578415125,2020-01-07 22:38:45+00:00,ed5cd52fb98da7fd907ad8b60889c916a55c6ade2ec115...,19.815141,-97.497945,122.400000
1310043,1578416452,2020-01-07 23:00:52+00:00,ed5cd52fb98da7fd907ad8b60889c916a55c6ade2ec115...,19.815141,-97.497945,149.600000
1310044,1578404374,2020-01-07 19:39:34+00:00,95495a83e7929cc1aced3a383cb9c01300463cad9e993f...,21.064956,-99.040763,112.000000
1310045,1578404383,2020-01-07 19:39:43+00:00,95495a83e7929cc1aced3a383cb9c01300463cad9e993f...,21.064960,-99.040760,112.000000


### Localización en ZVM

In [11]:
big_union["geometry"] = big_union[["latitude", "longitude"]] \
    .apply(lambda x : Point(x["longitude"], x["latitude"]), axis=1)

In [12]:
gdf_L = GeoDataFrame(big_union, geometry='geometry', crs="EPSG:4485")
gdf_R = GeoDataFrame(ZVM, geometry='geometry', crs="EPSG:4485")

joined = gdf_L.sjoin(gdf_R, how="left")

In [13]:
joined.crs

<Derived Projected CRS: EPSG:4485>
Name: Mexico ITRF92 / UTM zone 12N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Mexico between 114°W and 108°W, onshore and offshore.
- bounds: (-114.0, 15.09, -108.0, 32.27)
Coordinate Operation:
- name: UTM zone 12N
- method: Transverse Mercator
Datum: Mexico ITRF92
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [17]:
joined = joined.drop(columns=["geometry", "index_right"])

In [25]:
pings_in_ZVM = duck.sql("""
SELECT *
FROM joined
WHERE cve_mun IS NOT NULL
""").df()

In [30]:
len(pings_in_ZVM["cve_mun"].unique())

76

In [32]:
pings_in_ZVM

,utc_timestamp,cdmx_datetime,caid,latitude,longitude,horizontal_accuracy,cve_zm,cve_mun
0,1578346604,2020-01-07 03:36:44+00:00,38b2d1fecd656d2a69e444dcc8550aa0984f159e08426d...,19.676037,-98.991331,128.9,09.01,15081
1,1578350144,2020-01-07 04:35:44+00:00,9779f1452cce0914eeffa2be05644e44c86990106367d0...,19.681130,-98.899210,104.0,09.01,15092
2,1578350709,2020-01-07 04:45:09+00:00,9779f1452cce0914eeffa2be05644e44c86990106367d0...,19.681130,-98.899210,110.0,09.01,15092
3,1578352727,2020-01-07 05:18:47+00:00,90288b8d5205f051a3c5739145cee49f10665b3419c801...,19.710913,-98.817723,116.1,09.01,15075
4,1578352754,2020-01-07 05:19:14+00:00,becdf72416d1f0fa15e3ba2384dcdec494542d3de243ad...,19.591302,-99.025113,100.0,09.01,15033
...,...,...,...,...,...,...,...,...
683745,1578410832,2020-01-07 21:27:12+00:00,94170a767dd818bc03d7e9592d549157cf183af24143e0...,19.431026,-99.090817,165.0,09.01,09017
683746,1578407040,2020-01-07 20:24:00+00:00,fb3df61322061119387373b93d492f774ca528489e1b44...,19.403640,-99.122484,118.0,09.01,09006
683747,1578410873,2020-01-07 21:27:53+00:00,f6c001d76b80c050d655060852d34f27462d7bce7fa224...,19.403325,-99.121485,165.0,09.01,09006
683748,1578411864,2020-01-07 21:44:24+00:00,95670856f6795e1a3af44d296cf473f046233324088793...,19.414576,-99.130556,200.0,09.01,09015
